In [ ]:
from keras import models, layers, losses, optimizers, activations, metrics, regularizers
import pandas as pd
import numpy as np

Fetching the data

In [ ]:
df = pd.read_csv("data.csv")
print(df)

In [ ]:

# inputs = 
# expected_outputs = 
# expected_outputs = expected_outputs[:, 0]
# matrix_spikes_inputs = pd.read_csv(
#     "data.csv", sep=",", header=0, usecols=[4, 4]
# ).values
# matrix_spikes_inputs = matrix_spikes_inputs[:, 0]

# new_inputs = np.zeros(((inputs.shape[0]), 16))
# for index, spikes in enumerate(matrix_spikes_inputs):
#     spike_array = []
# for spike in spikes:
#     if spike == "0":
#         spike_array.append(0)
#     elif spike == "1":
#         spike_array.append(1)

# new_array = np.append(inputs[index], spike_array)
# new_inputs[index] = new_array


Creating neural network

In [ ]:
model = models.Sequential()

model.add(layers.Dense(32, activation=activations.linear, input_shape=(16,)))
model.add(layers.LeakyReLU(0.01))

model.add(layers.Dense(32, activation=activations.linear))
model.add(layers.LeakyReLU(0.01))

model.add(layers.Dense(1, activation=activations.sigmoid))


Model Training

In [ ]:
NUM_EPOCHS = 300 
model.compile(
    optimizer=optimizers.Adam(),
    loss=losses.BinaryCrossentropy(),
    metrics=metrics.BinaryAccuracy(),
)
history = model.fit(inputs, expected_outputs, epochs=NUM_EPOCHS, verbose=3)

Evaluate and save model


In [ ]:
model.save("my_model")